In [ ]:
"""
You are given nums, an array of positive integers of size 2 * n. You must perform n operations on this array.

In the ith operation (1-indexed), you will:

Choose two elements, x and y.
Receive a score of i * gcd(x, y).
Remove x and y from nums.
Return the maximum score you can receive after performing n operations.

The function gcd(x, y) is the greatest common divisor of x and y.

 

Example 1:
    Input: nums = [1,2]
    Output: 1
    Explanation: The optimal choice of operations is:
    (1 * gcd(1, 2)) = 1

Example 2:
    Input: nums = [3,4,6,8]
    Output: 11
    Explanation: The optimal choice of operations is:
    (1 * gcd(3, 6)) + (2 * gcd(4, 8)) = 3 + 8 = 11

Example 3:
    Input: nums = [1,2,3,4,5,6]
    Output: 14
    Explanation: The optimal choice of operations is:
    (1 * gcd(1, 5)) + (2 * gcd(2, 4)) + (3 * gcd(3, 6)) = 1 + 4 + 9 = 14


Constraints:
    1 <= n <= 7
    nums.length == 2 * n
    1 <= nums[i] <= 106
    
TIP:
    1. Maximize so DP on bruteforce
    2. Can any math work? We tried, assumptions for max gcd, but failed.
    3. Coming back to naive approache --- may work because of smaller value of n.
    4. Complexity --> O(n^n)
    5. Add caching --> O(2^n*n^2*log(K))
    6. Cache-key bit-mask of pair used so far.
"""
from typing import List

def gcd(x, y):
    while(y):
        x, y = y, x % y
    return abs(x)

## Finally --- hard question, bit-mask DP
# O(n^2 * 2^n * log(K))

class Solution:
    def maxScore(self, nums:List[int]) -> int:
        scores = {}
        nl = len(nums)
        def operate(op, mask):
            if mask in scores:
                return scores[mask]
            scores[mask] = 0
            for i in range(nl):
                if (1<<i) & mask:
                    continue
                for j in range(i+1, nl):
                    if (1<<j) & mask:
                        continue
                    new_mask = mask | (1<<i) | (1<<j)
                    score = op * gcd(nums[i], nums[j])
                    scores[mask] = max(
                        scores[mask],
                        score + operate(op+1, new_mask)
                    )
            return scores[mask]
        return operate(1, 0)


In [ ]:
## Wrong ----
## Assumption that gcd_i1_i2<i1, i2> if highest, should always be picked up.
## i1 and i2 can have separate pairs which may come out larger than 
# gcd_i1_i2 + gcd_i2_x + gcd_i1_y < gcd_i1_a + gcd_i1_b + gcd_x_p + gcd_y_q
from heapq import heapify, heappop
def gcd(a, b):
    while b:
        a, b = b, a%b
    return abs(a)

class Solution:
    def maxScore(self, nums: List[int]) -> int:
        gcds = []
        nl   = len(nums)
        n    = nl // 2
        for idx1 in range(nl):
            for idx2 in range(idx1+1, nl):
                gcds.append((-gcd(nums[idx1], nums[idx2]), (idx1, idx2)))
        heapify(gcds)
        print(gcds)
        score = []
        seen  = set()
        while len(score) < n:
            curr, (i1, i2) = heappop(gcds)
            if i1 in seen or i2 in seen:
                continue
            seen.add(i1)
            seen.add(i2)
            score.append(-curr)
        print(score)
        return sum([p*score[n-p] for p in range(n, 0, -1)])

In [1]:
# Sort, gcd max from top element assumption !!
# Wrong assumption --- 10, 15, 30, 40
class Solution:
    def maxScore(self, nums: List[int]) -> int:
        pairs = sorted(nums, reverse=True)
        visited = set()
        score = []
        rest  = []
        print(pairs)
        for idx1, num1 in enumerate(pairs):
            if idx1 in visited:
                continue
            curr = 1
            mark = None
            offset = idx1+1
            for idx2, num2 in enumerate(pairs[offset:]):
                x = gcd(num1, num2)
                if x > curr:
                    curr = x
                    mark = idx2+offset
            print(f"n1={idx1}, n2={mark}, gcd={curr}")
            if mark:
                score += [curr]
                visited.add(mark)
            else:
                rest += [1]
            visited.add(idx1)
        total = score + rest[:len(rest)//2]
        n = len(nums)//2
        result= sum([p*total[n-p] for p in range(n, 0, -1)])
        return result

In [ ]:

# Wrong ---> a num used multiple times
from heapq import heapify, heappush, heappop

def gcd(a, b):
    while b:
        a, b = b, a%b
    return abs(a)

class Solution:
    def maxScore(self, nums: List[int]) -> int:
        gcd_heap = []
        nl = len(nums)
        n  = nl//2
        visited = set()
        for idx1, num1 in enumerate(nums):
            for idx2, num2 in enumerate(nums[idx1:]):
                x = gcd(num1, num2)
                if len(gcd_heap) < n:
                    heappush(gcd_heap, (x))
                    visited.add(idx1)
                    visited.add(idx2)
                else:
                    if x > gcd_heap[-1]:
                        heappop(gcd_heap)
                        heappush(gcd_heap, x)
        score = 0    
        for p in range(n, 0, -1):
            score += p * heappop(gcd_heap)
        return score

In [6]:
gcd(697035, 575458)

1